In [11]:
# IMPORTING LIBRARIES
import pandas as pd
import numpy as np
import seaborn as sns

In [12]:
#SETTINGS
sns.set(rc={'figure.figsize':(20,20)})

In [13]:
#READ DATA
sample_data = pd.read_excel('data\SAMPLE_DATA.xlsx', header = 1)

In [14]:
#EXTRACT EXPLANATION 
explantion = sample_data['Variables']
sample_data = sample_data.drop(['Variables'], axis=1)

In [15]:
#TRANSPOSE DATA
transpose_sample_data = sample_data.T

In [18]:
#ADD HEADER
main_data = transpose_sample_data.rename(columns=transpose_sample_data.iloc[0]).drop(transpose_sample_data.index[0])

In [19]:
main_df = main_data.astype(float)

In [20]:
main_df.head(5)

,BRAND_A_VOLUME,BRAND_A_PRICE,COMPETITOR_A_PRICE,COMPETITOR_B_PRICE,COMPETITOR_C_PRICE,COMPETITOR_D_PRICE,COMPETITOR_E_PRICE,BRAND_A_New product,COMPETITOR_New product,BRAND_A_Insert,...,BRAND_A_OTHER_CATEGORY_1_GRP,BRAND_A_OTHER_CATEGORY_2_GRP,COMPETITOR_B_TOTAL_GRP,COMPETITOR_C_TOTAL_GRP,COMPETITOR_D_TOTAL_GRP,COMPETITOR_E_TOTAL_GRP,COMPETITOR_F_TOTAL_GRP,Real_Private_Consumption,BRAND_A_NDP,Teshir_Orani_BRAND_A
2016-05-02 00:00:00,35232.8,12.593379,11.89,15.37,13.35,11.74,0.0,0.0,0.0,0.0,...,0.0,72.0,0.0,0.0,0.0,0.0,0.0,964.074,0.0,0.0
2016-05-09 00:00:00,155680.4,9.306888,9.51,9.65,9.94,8.36,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,964.074,0.0,0.0
2016-05-16 00:00:00,34356.6,12.539075,12.14,15.17,13.04,11.60,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,964.074,0.0,0.0
2016-05-23 00:00:00,34145.7,12.915243,12.69,15.61,13.49,11.61,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,964.074,0.0,0.0
2016-05-30 00:00:00,35328.5,12.678149,12.31,15.47,13.67,12.17,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,964.074,0.0,0.0
